In [1]:
import requests
import pandas as pd
import time

In [ ]:
# Replace these with your actual Riot ID and tag inside of the quotations
riot_id = ""
tag_line = ""

# Your Riot API key from https://developer.riotgames.com/
api_key = ""

headers = {
    "X-Riot-Token": api_key
}

In [3]:
account_url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{riot_id}/{tag_line}"

response = requests.get(account_url, headers=headers)
if response.status_code == 200:
    puuid = response.json()["puuid"]
    print("PUUID:", puuid)
else:
    print("❌ Error fetching account info:", response.status_code, response.json())


PUUID: mb4w1nbnQWi4zz5XlhYVu2yio333k82i-7AceTp8fRgkaYRP3POv4fpuAzze5-YzRqN4_yfILwZsHQ


In [4]:
def get_arena_match_ids(puuid, start=0, count=100, max_matches=1000):
    match_ids = []
    print("Fetching Arena matches...")
    while len(match_ids) < max_matches:
        url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=1700&start={start}&count={count}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            batch = response.json()
            if not batch:
                break
            match_ids.extend(batch)
            print(f"Fetched {len(match_ids)} total matches...")
            start += count
            time.sleep(1)
        else:
            print("❌ Error:", response.status_code, response.json())
            break
    return match_ids

match_ids = get_arena_match_ids(puuid)


Fetching Arena matches...
Fetched 100 total matches...
Fetched 200 total matches...
Fetched 300 total matches...
Fetched 400 total matches...
Fetched 437 total matches...


In [5]:
def get_match_data(match_id, puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch match: {match_id}")
        return None

    data = response.json()
    try:
        for player in data["info"]["participants"]:
            if player["puuid"] == puuid:
                return {
                    "match_id": match_id,
                    "champion": player["championName"],
                    "placement": player["placement"],
                    "game_mode": data["info"]["gameMode"],
                    "queue_id": data["info"]["queueId"],
                    "game_end": data["info"]["gameEndTimestamp"]
                }
    except KeyError:
        print(f"Missing keys in match: {match_id}")
    return None

rows = []
for idx, match_id in enumerate(match_ids):
    match = get_match_data(match_id, puuid)
    if match:
        rows.append(match)
    time.sleep(1)  # Avoid hitting rate limits

print(f"Total matches retrieved: {len(rows)}")


Total matches retrieved: 437


In [6]:
df = pd.DataFrame(rows)
df["game_end"] = pd.to_datetime(df["game_end"], unit='ms')
df = df.sort_values("game_end", ascending=False)
df.reset_index(drop=True, inplace=True)
df.head()


,match_id,champion,placement,game_mode,queue_id,game_end
0,NA1_5337927464,Karthus,1,CHERRY,1700,2025-08-03 00:17:01.055
1,NA1_5337910469,Ahri,5,CHERRY,1700,2025-08-02 23:52:09.272
2,NA1_5337494529,Urgot,1,CHERRY,1700,2025-08-02 08:29:30.541
3,NA1_5337483961,Ahri,4,CHERRY,1700,2025-08-02 08:04:23.034
4,NA1_5337468710,Xerath,7,CHERRY,1700,2025-08-02 07:45:32.795


In [80]:
# Filter to Arena games from June 26, 2024 and onward
arena_start = pd.Timestamp("2024-06-26")
current_arena_df = df[df["game_end"] >= arena_start].reset_index(drop=True)
current_arena_df.head()

,match_id,champion,placement,game_mode,queue_id,game_end
0,NA1_5337927464,Karthus,1,CHERRY,1700,2025-08-03 00:17:01.055
1,NA1_5337910469,Ahri,5,CHERRY,1700,2025-08-02 23:52:09.272
2,NA1_5337494529,Urgot,1,CHERRY,1700,2025-08-02 08:29:30.541
3,NA1_5337483961,Ahri,4,CHERRY,1700,2025-08-02 08:04:23.034
4,NA1_5337468710,Xerath,7,CHERRY,1700,2025-08-02 07:45:32.795


In [82]:
# Keep only champion and placement
champ_df = current_arena_df[["champion", "placement"]]

# Group by champion and calculate metrics
champion_summary = champ_df.groupby("champion").agg(
    games_played=("placement", "count"),
    average_placement=("placement", "mean"),
    got_1st_place=("placement", lambda x: (x == 1).any())
).reset_index()

# Round the average placement
champion_summary["average_placement"] = champion_summary["average_placement"].round(2)

champion_summary.head()

,champion,games_played,average_placement,got_1st_place
0,Aatrox,2,4.50,True
1,Ahri,11,4.73,False
2,Akali,4,5.00,False
3,Akshan,2,2.00,True
4,Alistar,2,2.00,True


In [84]:
# Filter for champions that have NOT gotten 1st place
no_1st_place = champion_summary[champion_summary["got_1st_place"] == False]

# Sort by best (lowest) average placement
no_1st_place = no_1st_place.sort_values("average_placement").reset_index(drop=True)

no_1st_place.head(10)  # Top 10 champs closest to a win

,champion,games_played,average_placement,got_1st_place
0,Milio,1,2.00,False
1,Kassadin,1,2.00,False
2,Karma,1,2.00,False
3,Warwick,2,2.00,False
4,Trundle,1,2.00,False
5,Ekko,1,2.00,False
6,Rell,1,2.00,False
7,Gangplank,1,2.00,False
8,Yasuo,3,2.33,False
9,Yorick,2,2.50,False


In [92]:
# Total matches played
total_matches = current_arena_df.shape[0]

# Arena Legend progress = unique champions played
unique_champs_played = champion_summary.shape[0]
arena_legend_progress = f"{champions_with_win.shape[0]}/171"

# Arena God progress = champions with at least one 1st place finish
champions_with_win = champion_summary[champion_summary["got_1st_place"] == True]
arena_god_progress = f"{champions_with_win.shape[0]}/60"

# Unique champions played
unique_champs_played_progress = f"{unique_champs_played}/171"

# Top 10 champions by matches played
top_played_champs = champion_summary.sort_values("games_played", ascending=False).head(10)

# Champions that took the most matches before first win
# Filter for champions with at least one win and sort by games played
most_matches_for_win = champions_with_win.sort_values("games_played", ascending=False).head(10)

# Champions won on first try (only 1 game played and it's a win)
first_try_wins = champions_with_win[champions_with_win["games_played"] == 1]["champion"].tolist()

In [94]:
summary =f""""""

In [96]:

# Create Reddit summary
summary = f"""
Total matches played: {total_matches}
Arena Legend progress: {arena_legend_progress}
Arena God progress: {arena_god_progress}
Unique champions played: {unique_champs_played_progress}

Champions with the most matches played (Top 10):
{top_played_champs[['champion', 'games_played']].to_string(index=False)}

Champions that took the most matches before first win (Top 10):
{most_matches_for_win[['champion', 'games_played']].to_string(index=False)}

Champions won on first try:
{', '.join(first_try_wins)}
"""

print(summary)


Total matches played: 374
Arena Legend progress: 60/171
Arena God progress: 60/60
Unique champions played: 167/171

Champions with the most matches played (Top 10):
champion  games_played
    Ahri            11
  Thresh            10
 Smolder             9
  Ezreal             8
    Jhin             8
    Ashe             7
   Briar             7
    Ornn             7
  Veigar             6
 Kindred             6

Champions that took the most matches before first win (Top 10):
 champion  games_played
  Smolder             9
     Jhin             8
   Ezreal             8
     Ornn             7
TahmKench             5
    Leona             5
     Jinx             5
   Aurora             5
    Braum             5
   KogMaw             4

Champions won on first try:
AurelionSol, Azir, Belveth, DrMundo, Evelynn, Ivern, Lux, Malphite, Renata, Shaco, Soraka, Sylas, Taliyah, Taric, Tryndamere, Vex, Viego, Ziggs

